### 1. Реалізувати підпрограми із необхідними математичними операціями: обчисленням оберненого елементу за модулем із використанням розширеного алгоритму Евкліда, розв’язуванням лінійних порівнянь. При розв’язуванні порівнянь потрібно коректно обробляти випадок із декількома розв’язками, повертаючи їх усі.

In [26]:
import math
from collections import Counter
import pandas as pd
import os

In [27]:
def rozshyrenyy_nsd(a, b):
    if b == 0:
        return a, 1, 0
    nsd, x1, y1 = rozshyrenyy_nsd(b, a % b)
    x = y1
    y = x1 - (a // b) * y1
    return nsd, x, y

def obernyty_mod(a, m):
    nsd, x, _ = rozshyrenyy_nsd(a, m)
    if nsd != 1:
        raise ValueError(f"{a} ne maie obernenoho elementa za modulem {m}")
    return x % m

def vyrishyty_liniyne_spivvidnoshennya(a, b, m):
    nsd, x, _ = rozshyrenyy_nsd(a, m)

    if b % nsd != 0:
        return 0

    a_skorot = a // nsd
    b_skorot = b // nsd
    m_skorot = m // nsd

    x0 = (obernyty_mod(a_skorot, m_skorot) * b_skorot) % m_skorot

    rozvyazky = [(x0 + k * m_skorot) % m for k in range(nsd)]
    return rozvyazky

print(vyrishyty_liniyne_spivvidnoshennya(2, 4, 6))


[2, 5]


### 2. За допомогою програми обчислення частот біграм, яка написана в ході виконання комп’ютерного практикуму №1, знайти 5 найчастіших біграм запропонованого шифртексту (за варіантом).

In [28]:
import math
from collections import Counter
import pandas as pd

def obchyslyty_chastoty_bigram(tekst):
    bigramy = [tekst[i:i+2].replace(' ', '_') for i in range(0, len(tekst) - 1, 2)]
    
    zahalna_kilkist_bigram = len(bigramy)
    kilkist_bigram = Counter(bigramy)

    chastoty = {bigram: kilkist / zahalna_kilkist_bigram for bigram, kilkist in kilkist_bigram.items()}
    
    return chastoty, kilkist_bigram, zahalna_kilkist_bigram

def obchyslyty_entropiyu(chastoty):
    entropiya = (-sum(chastota * math.log2(chastota) for chastota in chastoty.values() if chastota > 0))/2
    return entropiya

with open('09.txt', 'r', encoding='utf-8') as file:
    tekst = file.read().replace('\n','')

unikalni_symvoly = sorted(set(tekst.replace(' ', '_')))

chastoty_bigram, kilkist_bigram, zahalna_kilkist_bigram = obchyslyty_chastoty_bigram(tekst)

matrytsya_bigram = pd.DataFrame(0.0, index=unikalni_symvoly, columns=unikalni_symvoly)

for bigram, chastota in chastoty_bigram.items():
    if len(bigram) == 2:
        pershyy_symvol = bigram[0]
        druhyy_symvol = bigram[1]
        matrytsya_bigram.at[pershyy_symvol, druhyy_symvol] = chastota



entropiya_H2 = obchyslyty_entropiyu(chastoty_bigram)
top_5_bigram_chastota = kilkist_bigram.most_common(5)
print(f"Матриця частот біграм без перетинів успішно збережена в 'chastoty_bigram_bez_peretyniv.xlsx'.")
print(f"Ентропія H2: {entropiya_H2:.6f}")
print(top_5_bigram_chastota)



Матриця частот біграм без перетинів успішно збережена в 'chastoty_bigram_bez_peretyniv.xlsx'.
Ентропія H2: 4.076883
[('ээ', 74), ('вд', 57), ('чф', 56), ('цг', 56), ('гн', 56)]


### 3. Перебрати можливі варіанти співставлення частих біграм мови та частих біграм шифртексту (розглядаючи пари біграм із п’яти найчастіших). Для кожного співставлення знайти можливі кандидати на ключ (a,b) шляхом розв’язання системи (1).


In [29]:
top_5_bigram = [x[0] for x in top_5_bigram_chastota]

def pary_naichastishykh_bigram(spysok, stan_pary=True):
    pary_bigram = []
    for i in range(len(spysok)):
        for j in range(i + 1, len(spysok)): 
            pary_bigram.append((spysok[i], spysok[j]))
    return pary_bigram

populyarni_pary_tekstu = pary_naichastishykh_bigram(top_5_bigram)
print(populyarni_pary_tekstu)

populyarni_pary_movy = pary_naichastishykh_bigram(['ст', 'но', 'то', 'на', 'ен'])
print(populyarni_pary_movy)

populyarni_pary = pary_naichastishykh_bigram(top_5_bigram + ['ст', 'но', 'то', 'на', 'ен'])
print(populyarni_pary)


[('ээ', 'вд'), ('ээ', 'чф'), ('ээ', 'цг'), ('ээ', 'гн'), ('вд', 'чф'), ('вд', 'цг'), ('вд', 'гн'), ('чф', 'цг'), ('чф', 'гн'), ('цг', 'гн')]
[('ст', 'но'), ('ст', 'то'), ('ст', 'на'), ('ст', 'ен'), ('но', 'то'), ('но', 'на'), ('но', 'ен'), ('то', 'на'), ('то', 'ен'), ('на', 'ен')]
[('ээ', 'вд'), ('ээ', 'чф'), ('ээ', 'цг'), ('ээ', 'гн'), ('ээ', 'ст'), ('ээ', 'но'), ('ээ', 'то'), ('ээ', 'на'), ('ээ', 'ен'), ('вд', 'чф'), ('вд', 'цг'), ('вд', 'гн'), ('вд', 'ст'), ('вд', 'но'), ('вд', 'то'), ('вд', 'на'), ('вд', 'ен'), ('чф', 'цг'), ('чф', 'гн'), ('чф', 'ст'), ('чф', 'но'), ('чф', 'то'), ('чф', 'на'), ('чф', 'ен'), ('цг', 'гн'), ('цг', 'ст'), ('цг', 'но'), ('цг', 'то'), ('цг', 'на'), ('цг', 'ен'), ('гн', 'ст'), ('гн', 'но'), ('гн', 'то'), ('гн', 'на'), ('гн', 'ен'), ('ст', 'но'), ('ст', 'то'), ('ст', 'на'), ('ст', 'ен'), ('но', 'то'), ('но', 'на'), ('но', 'ен'), ('то', 'на'), ('то', 'ен'), ('на', 'ен')]


In [30]:
def pary_u_chysla(spysok, stan_pary=True):
    abetka = 'абвгдежзийклмнопрстуфхцчшщьыэюя'
    chysla_pary = []
    if stan_pary:
        for para in spysok:
            x1 = abetka.index(para[0][0]) * 31 + abetka.index(para[0][1])
            x2 = abetka.index(para[1][0]) * 31 + abetka.index(para[1][1])
            chysla_pary.append((x1, x2))
    else:
        for elem in spysok:
            x1 = abetka.index(elem[0]) * 31 + abetka.index(elem[1])
            chysla_pary.append(x1)
    return chysla_pary

chysla_pary_tekstu = pary_u_chysla(populyarni_pary_tekstu)
chysla_pary_movy = pary_u_chysla(populyarni_pary_movy)
chysla_pary = pary_u_chysla(populyarni_pary)

print(chysla_pary_tekstu)
print(chysla_pary_movy)
print(chysla_pary)


[(896, 66), (896, 733), (896, 685), (896, 106), (66, 733), (66, 685), (66, 106), (733, 685), (733, 106), (685, 106)]
[(545, 417), (545, 572), (545, 403), (545, 168), (417, 572), (417, 403), (417, 168), (572, 403), (572, 168), (403, 168)]
[(896, 66), (896, 733), (896, 685), (896, 106), (896, 545), (896, 417), (896, 572), (896, 403), (896, 168), (66, 733), (66, 685), (66, 106), (66, 545), (66, 417), (66, 572), (66, 403), (66, 168), (733, 685), (733, 106), (733, 545), (733, 417), (733, 572), (733, 403), (733, 168), (685, 106), (685, 545), (685, 417), (685, 572), (685, 403), (685, 168), (106, 545), (106, 417), (106, 572), (106, 403), (106, 168), (545, 417), (545, 572), (545, 403), (545, 168), (417, 572), (417, 403), (417, 168), (572, 403), (572, 168), (403, 168)]


In [31]:
from itertools import combinations

def znajty_kluchi_spilni(spilni_pary, m):
    mod_m2 = m ** 2  # m^2 — кількість можливих числових біграм
    kandidaty = []
    
    # Перебираємо всі комбінації двох пар біграм
    for para1, para2 in combinations(spilni_pary, 2):
        x1, x2 = para1
        y1, y2 = para2

        # Обчислюємо різниці між біграмами
        delta_x = (x1 - x2) % mod_m2
        delta_y = (y1 - y2) % mod_m2

        # Знаходимо можливі значення a
        a_kandydaty = vyrishyty_liniyne_spivvidnoshennya(delta_x, delta_y, mod_m2)

        if isinstance(a_kandydaty, list):  # Якщо знайдено кілька розв'язків
            for a in a_kandydaty:
                b = (y1 - a * x1) % mod_m2  # Обчислюємо b
                kandidaty.append((a, b))  # Додаємо пару (a, b) до кандидатів
        else:
            continue
    
    return kandidaty

m=31

# Пошук ключів
kluchi_spilni = znajty_kluchi_spilni(chysla_pary, m)

# Виведення ключів
for item in kluchi_spilni:
    print(f"Klyuch (a, b): {item}")


Klyuch (a, b): (703, 463)
Klyuch (a, b): (798, 872)
Klyuch (a, b): (82, 460)
Klyuch (a, b): (34, 223)
Klyuch (a, b): (928, 673)
Klyuch (a, b): (401, 53)
Klyuch (a, b): (275, 512)
Klyuch (a, b): (640, 212)
Klyuch (a, b): (702, 529)
Klyuch (a, b): (797, 938)
Klyuch (a, b): (81, 526)
Klyuch (a, b): (33, 289)
Klyuch (a, b): (927, 739)
Klyuch (a, b): (400, 119)
Klyuch (a, b): (274, 578)
Klyuch (a, b): (639, 278)
Klyuch (a, b): (95, 181)
Klyuch (a, b): (340, 730)
Klyuch (a, b): (292, 493)
Klyuch (a, b): (225, 943)
Klyuch (a, b): (659, 323)
Klyuch (a, b): (533, 782)
Klyuch (a, b): (898, 482)
Klyuch (a, b): (245, 273)
Klyuch (a, b): (197, 36)
Klyuch (a, b): (130, 486)
Klyuch (a, b): (564, 827)
Klyuch (a, b): (438, 325)
Klyuch (a, b): (803, 25)
Klyuch (a, b): (913, 830)
Klyuch (a, b): (846, 319)
Klyuch (a, b): (319, 660)
Klyuch (a, b): (193, 158)
Klyuch (a, b): (558, 819)
Klyuch (a, b): (894, 34)
Klyuch (a, b): (367, 375)
Klyuch (a, b): (241, 834)
Klyuch (a, b): (606, 534)
Klyuch (a, b): (434, 

### 4. Для кожного кандидата на ключ дешифрувати шифртекст. Якщо шифртекст не є змістовним текстом російською мовою, відкинути цього кандидата.

In [32]:
import os
import numpy as np
from collections import Counter

def chastota_simvoliv(tekst, abetka):
    tekst = tekst.lower()
    kilkist_simvoliv = Counter([simvol for simvol in tekst if simvol in abetka])
    return kilkist_simvoliv

def chastota_bigram(tekst, abetka):
    bigramy = [tekst[i:i+2] for i in range(len(tekst)-1) if tekst[i:i+2] in abetka]
    kilkist_bigram = Counter(bigramy)
    return kilkist_bigram

def perevirka_tekstu(tekst, abetka):
    chastota_lyter = chastota_simvoliv(tekst, abetka)
    chastota_bigramu = chastota_bigram(tekst, abetka)
    naipopulyarnishi_lytery = sorted(chastota_lyter, key=chastota_lyter.get, reverse=True)[:5]
    naimensh_populyarni_lytery = sorted(chastota_lyter, key=chastota_lyter.get, reverse=True)[-5:]
    populyarni_lytery = {'о', 'а', 'е'}
    ridkisni_lytery = {'ф', 'щ', 'ь'}
    spivpaly_populyarnykh = len(set(naipopulyarnishi_lytery) & populyarni_lytery)
    spivpaly_ridkisnykh = len(set(naimensh_populyarni_lytery) & ridkisni_lytery)
    if spivpaly_populyarnykh >= 1 and spivpaly_ridkisnykh >= 1:
        populyarni_bigramy = ['ст', 'но', 'ен', 'то', 'на']
        top_bigramy = sorted(chastota_bigramu, key=chastota_bigramu.get, reverse=True)[:6]
        spivpaly_bigram = len(set(top_bigramy) & set(populyarni_bigramy))
        if spivpaly_bigram >= 2:
            return True
    return False

def tekst_u_chyslovyi_masiv(tekst, m):
    abetka = "абвгдежзийклмнопрстуфхцчшщьыэюя"
    rezultat = []
    for i in range(0, len(tekst) - 1, 2):
        if tekst[i].lower() in abetka and tekst[i + 1].lower() in abetka:
            x1 = abetka.index(tekst[i].lower())
            x2 = abetka.index(tekst[i + 1].lower())
            rezultat.append(x1 * m + x2)
    return rezultat

def chyslovyi_masiv_u_tekst(chyslovyi_masiv, m):
    abetka = "абвгдежзийклмнопрстуфхцчшщьыэюя"
    rezultat = []
    for num in chyslovyi_masiv:
        x1 = num // m
        x2 = num % m
        rezultat.append(abetka[x1] + abetka[x2])
    return ''.join(rezultat)

def rozshyfruvaty(chyslovyi_masiv, a, b, m):
    mod_m2 = m**2
    a_inv = obernyty_mod(a, mod_m2)
    return [(a_inv * (num - b)) % mod_m2 for num in chyslovyi_masiv]

with open('09.txt', 'r', encoding='utf-8') as file:
    shyfrtekst = file.read().strip().replace('\n', '')

m = 31
chyslovyi_shyfrtekst = tekst_u_chyslovyi_masiv(shyfrtekst, m)

output_dir = "rozshyfrovanі_teksty"
os.makedirs(output_dir, exist_ok=True)

for para in kluchi_spilni:
    a, b = para
    try:
        rozshyfrovanі_chysla = rozshyfruvaty(chyslovyi_shyfrtekst, a, b, m)
        rozshyfrovanij_tekst = chyslovyi_masiv_u_tekst(rozshyfrovanі_chysla, m)

        if perevirka_tekstu(rozshyfrovanij_tekst, "абвгдежзийклмнопрстуфхцчшщьыэюя"):
            output_filename = f"{output_dir}/rozshyfrovanij_a{a}_b{b}.txt"
            with open(output_filename, 'w', encoding='utf-8') as output_file:
                output_file.write(rozshyfrovanij_tekst)
            print(f"Rozshyfrovanij tekst zberezheno u fail: {output_filename}")
        else:
            print(f"Klyuch (a={a}, b={b}) ne proishov perevirku.")
    except ValueError as e:
        print(f"Propushcheno klyuch (a={a}, b={b}) - {e}")


Klyuch (a=703, b=463) ne proishov perevirku.
Klyuch (a=798, b=872) ne proishov perevirku.
Klyuch (a=82, b=460) ne proishov perevirku.
Klyuch (a=34, b=223) ne proishov perevirku.
Klyuch (a=928, b=673) ne proishov perevirku.
Klyuch (a=401, b=53) ne proishov perevirku.
Klyuch (a=275, b=512) ne proishov perevirku.
Klyuch (a=640, b=212) ne proishov perevirku.
Klyuch (a=702, b=529) ne proishov perevirku.
Klyuch (a=797, b=938) ne proishov perevirku.
Klyuch (a=81, b=526) ne proishov perevirku.
Klyuch (a=33, b=289) ne proishov perevirku.
Klyuch (a=927, b=739) ne proishov perevirku.
Klyuch (a=400, b=119) ne proishov perevirku.
Klyuch (a=274, b=578) ne proishov perevirku.
Klyuch (a=639, b=278) ne proishov perevirku.
Klyuch (a=95, b=181) ne proishov perevirku.
Klyuch (a=340, b=730) ne proishov perevirku.
Klyuch (a=292, b=493) ne proishov perevirku.
Klyuch (a=225, b=943) ne proishov perevirku.
Klyuch (a=659, b=323) ne proishov perevirku.
Klyuch (a=533, b=782) ne proishov perevirku.
Klyuch (a=898, b